In [7]:
import dspy
import halide as hl
import numpy as np
import cv2

# Use a strong base model (can be GPT, Llama, etc.)
lm = dspy.LM(
    "openai/llama3.1:8b",  # model name is arbitrary but should match your endpoint
    api_base="http://172.27.21.160:11434/v1",  # your local server
    api_key="ollama",  # if your server requires this token
    model_type="chat"  # use 'chat' for instruct-style models
)

dspy.configure(lm=lm)

In [ ]:
import dspy
import json

# Load reference examples from JSON
with open("halide_examples.json") as f:
    raw_examples = json.load(f)

reference_examples = [
    dspy.Example(
        prompt=ex["prompt"],
        halide_code=ex["halide_code"],
        test_cases=""  # we don't separate test cases anymore; it's inside halide_code JSON
    )
    for ex in raw_examples
]

class HalideCodeGen(dspy.Signature):
    """Generate full Halide code and a test case from a prompt."""
    prompt = dspy.InputField(desc="Natural language description of the image processing task.")
    halide_code = dspy.OutputField(desc="Full Halide code that reads input.png and writes output.png.")
    test_cases = dspy.OutputField(
        desc=(
            "A JSON array of multiple test cases. Each test case should use small clean numbers "
            "(e.g., 0.0, 0.25, 0.5, 0.75, 1.0) with limited decimals for easy verification. "
            "Each element must contain: 'format', 'dtype', 'shape', 'input', 'expected_output', and 'notes'."
        )
    )


class HalidePipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.gen = dspy.Predict(HalideCodeGen)

    def forward(self, prompt):
        return self.gen(prompt=prompt)

# Initialize DSPy with your model (e.g., OpenAI, Together, etc.)
# Example: dspy.settings.configure(openai_api_key="...", model="gpt-4")
from dspy.teleprompt import BootstrapFewShot

tele = BootstrapFewShot(metric=None)
refined_pipeline = tele.compile(HalidePipeline(), trainset=reference_examples)

  0%|          | 0/2 [00:00<?, ?it/s]2025/11/02 14:18:20 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'prompt': 'Write a halide program in python to Brighten an image by 2x', 'halide_code': "import halide as hl\nimport imageio\nimport numpy as np\n\n# Constructing Halide functions statically.\ninput = hl.ImageParam(hl.Float(32), 3)\nf = hl.Func('f')\nx, y, c = hl.Var('x'), hl.Var('y'), hl.Var('c')\n# Double the values and clamp them by 1.\nf[x, y, c] = hl.min(2 * input[x, y, c], 1.0)\n\n# Actually compiling/executing the Halide functions.\n# Setup the input by loading an image (Halide assumes Fortran ordering).\nimg_np = imageio.imread('input.png').astype(np.float32) / 255.0\n# Ensure shape is (height, width, channels) and memory is contiguous\nimg_np = np.ascontiguousarray(img_np)\n\nimg = hl.Buffer(img_np)\ninput.set(img)\n# Process the input by calling f.realize\noutput = f.realize([img.width(), img.height(), img.channels()])\n# Save the image to a 

Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.


In [2]:
import halide as hl
import imageio
import numpy as np

# Step 1: Load and normalize the input image
img_np = imageio.imread("input.png").astype(np.float32) / 255.0

# Step 2: Ensure shape is (height, width, channels) and memory is contiguous
if img_np.ndim == 2:
    img_np = img_np[:, :, np.newaxis]  # grayscale fallback
img_np = np.ascontiguousarray(img_np)

# Step 3: Transpose to match Halide's layout: [width, height, channels]
img_for_halide = np.transpose(img_np, (2,0,1)).copy()
input_buf = hl.Buffer(img_for_halide)

# Step 4: Declare Halide Vars and ImageParam
x, y, c = hl.Var("x"), hl.Var("y"), hl.Var("c")
input = hl.ImageParam(hl.Float(32), 3)
input.set(input_buf)

# Step 5: Clamp edges to avoid out-of-bounds access
clamped = hl.Func("clamped")
clamped[x, y, c] = input[
    hl.clamp(x, 0, input_buf.width() - 1),
    hl.clamp(y, 0, input_buf.height() - 1),
    c
]

# Step 6: Horizontal blur (blur across X)
blur_x = hl.Func("blur_x")
blur_x[x, y, c] = (clamped[x - 1, y, c] + clamped[x, y, c] + clamped[x + 1, y, c]) / 3.0

# Step 7: Vertical blur (blur across Y)
blur_y = hl.Func("blur_y")
blur_y[x, y, c] = (blur_x[x, y - 1, c] + blur_x[x, y, c] + blur_x[x, y + 1, c]) / 3.0

# Step 8: Realize the output buffer using correct shape
output = blur_y.realize([input_buf.width(), input_buf.height(), input_buf.channels()])

# Step 9: Convert Halide Buffer to NumPy array and transpose back to HWC
output_np = np.array(output)
output_np = np.transpose(output_np, (1,2,0))  # [height, width, channels]

# Step 10: Save the blurred image
imageio.imsave("output.png", (output_np * 255.0).astype(np.uint8))

C:\Users\Cezan\AppData\Local\Temp\ipykernel_19444\3396105005.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_np = imageio.imread("input.png").astype(np.float32) / 255.0


In [5]:
# validator = HalideValidatorLoop(generator=HalidePipeline(), max_attempts=10)
# results = validator.run("Write a halide program in python to Brighten an image by 2x")

# for r in results:
#     print(f"\nAttempt {r['attempt']}:")
#     print("✅ Syntax OK:", r["syntax_ok"])
#     print("🧪 Correctness Index:", r["correctness_index"])
#     print("🔍 Expected vs Actual:", r.get("expected_pixel"), r.get("actual_pixel"))
#     if r["error"]:
#         print("❌ Error:", r["error"])


In [ ]:
import json, os, subprocess, tempfile, imageio, numpy as np

class HalideValidatorLoop:
    def __init__(self, generator, max_attempts=5):
        self.generator = generator
        self.max_attempts = max_attempts

    def _run_python(self, code: str, cwd: str, filename: str):
        path = os.path.join(cwd, filename)
        with open(path, "w") as f:
            f.write(code)
        try:
            subprocess.check_output(["python", path], stderr=subprocess.STDOUT, cwd=cwd)
            return True, None
        except subprocess.CalledProcessError as e:
            return False, e.output.decode()

    def _write_image(self, arr, path):
        arr = np.round(arr, 3)  # ✅ round to 3 decimal places for simplicity
        arr = np.clip(arr, 0.0, 1.0)
        imageio.imwrite(path, (arr * 255.0).astype(np.uint8))

    def _load_image(self, path):
        return imageio.imread(path).astype(np.float32) / 255.0

    def _compare(self, actual, expected, atol=0.01):
        if actual.shape != expected.shape:
            return 0.0, {"error": f"shape mismatch {actual.shape} vs {expected.shape}"}
        diff = np.abs(actual - expected)
        max_diff, mean_diff = float(diff.max()), float(diff.mean())
        passed = np.allclose(actual, expected, atol=atol)
        score = 1.0 if passed else max(0.0, 1.0 - mean_diff * 10)
        return score, {"max_diff": max_diff, "mean_diff": mean_diff}

    def _validate_multiple_cases(self, halide_code: str, test_cases_json: str):
        """Run halide_code for each test case in the JSON array."""
        try:
            cases = json.loads(test_cases_json)
            if isinstance(cases, dict):
                cases = [cases]
        except Exception as e:
            return {"syntax_ok": False, "error": f"Bad JSON: {e}", "results": []}

        all_results = []
        for idx, case in enumerate(cases):
            print(f"\nRunning Test Case #{idx+1}: {case.get('notes','(no notes)')}")
            with tempfile.TemporaryDirectory() as tmpdir:
                try:
                    input_arr = np.array(case["input"], dtype=case.get("dtype", "float32"))
                    expected_arr = np.array(case["expected_output"], dtype=case.get("dtype", "float32"))
                except Exception as e:
                    all_results.append({"idx": idx, "error": f"Bad arrays: {e}"})
                    continue

                # Write input and expected output images
                self._write_image(input_arr, os.path.join(tmpdir, "input.png"))
                self._write_image(expected_arr, os.path.join(tmpdir, "expected_output.png"))

                ok, err = self._run_python(halide_code, tmpdir, "halide_code.py")
                if not ok:
                    all_results.append({"idx": idx, "error": f"Halide code failed: {err}"})
                    continue

                try:
                    actual = self._load_image(os.path.join(tmpdir, "output.png"))
                    expected = self._load_image(os.path.join(tmpdir, "expected_output.png"))
                except Exception as e:
                    all_results.append({"idx": idx, "error": f"Load error: {e}"})
                    continue

                score, info = self._compare(actual, expected)
                result = {
                    "idx": idx,
                    "correctness_index": round(score, 3),
                    "max_diff": info.get("max_diff"),
                    "mean_diff": info.get("mean_diff"),
                    "expected_center": expected[1,1].tolist() if expected.shape[0]>=3 else expected.mean().tolist(),
                    "actual_center": actual[1,1].tolist() if actual.shape[0]>=3 else actual.mean().tolist(),
                }
                all_results.append(result)

                if score < 1.0:
                    print(f"Test #{idx+1} failed")
                    print("   Expected center:", result["expected_center"])
                    print("   Actual center:", result["actual_center"])
                    print("   Max diff:", result["max_diff"])
                else:
                    print(f"Test #{idx+1} passed perfectly")

        syntax_ok = all(r.get("error") is None for r in all_results)
        return {"syntax_ok": syntax_ok, "results": all_results}

    def run(self, prompt: str):
        """Generate code and validate multiple test cases."""
        print(f"\nPrompt: {prompt}")
        result = self.generator(prompt)
        halide_code = result.halide_code
        test_cases_json = result.test_cases

        print("\nGenerated Halide Code:\n", halide_code)
        print("\nGenerated Test Cases JSON:\n", test_cases_json)

        validation = self._validate_multiple_cases(halide_code, test_cases_json)

        print("\nValidation Summary:")
        for r in validation["results"]:
            if "error" in r:
                print(f"❌ Case {r['idx']+1}: Error - {r['error']}")
            else:
                print(f"Case {r['idx']+1}: Correctness {r['correctness_index']}, Mean diff {r['mean_diff']}")

        return validation


In [7]:
# validator = HalideValidatorLoop(generator=refined_pipeline, max_attempts=10)
# results = validator.run("Brighten a 3x3 RGB image by 2x using Halide. Generate test code that creates input.png and expected_output.png.")
# for r in results:
#     print(f"\nAttempt {r['attempt']}:")
#     print("✅ Syntax OK:", r["syntax_ok"])
#     print("🧪 Correctness Index:", r["correctness_index"])
#     print("🔍 Expected vs Actual:", r.get("expected_pixel"), r.get("actual_pixel"))
#     if r["error"]:
#         print("❌ Error:", r["error"])

In [ ]:
def _safe_json(obj):
    if isinstance(obj, (np.float32, np.float64)):
        return float(obj)
    if isinstance(obj, (np.ndarray,)):
        return obj.tolist()
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

def react_loop_with_code_feedback(base_prompt, pipeline, validator, max_rounds=3):
    """Feedback loop: the LLM refines code if any test fails."""
    last_error = None
    last_halide_code = None
    last_test_cases = None
    results = []

    for i in range(max_rounds):
        print(f"\n🔁 Round {i+1}")
        prompt = base_prompt
        if last_halide_code:
            prompt += f"\n\nPrevious Halide code:\n{last_halide_code}"
        if last_test_cases:
            prompt += f"\n\nPrevious test cases:\n{last_test_cases}"
        if last_error:
            prompt += f"\n\nError encountered:\n{last_error}\nPlease correct it."

        # Generate
        result = pipeline(prompt)
        halide_code = result.halide_code
        test_cases = result.test_cases

        validation = validator._validate_multiple_cases(halide_code, test_cases)

        results.append(validation)
        passed_all = all(
            (r.get("correctness_index", 0) == 1.0) for r in validation["results"] if "error" not in r
        )

        if passed_all:
            print("🎯 All test cases passed! Stopping early.")
            break

        last_halide_code = halide_code
        last_test_cases = test_cases
        last_error = json.dumps(validation, indent=2, default=_safe_json)

    return results


In [12]:
validator = HalideValidatorLoop(generator=refined_pipeline, max_attempts=1)

results = react_loop_with_code_feedback(
    base_prompt="Write a halide program in python to Brighten an image by 2x. Output as output.png. "
                "Also generate 3 structured test cases in JSON format as described.",
    pipeline=refined_pipeline,
    validator=validator,
    max_rounds=5
)


🔁 Round 1

Running Test Case #1: Brightening by 2Ã— and clamping values to 1.0


C:\Users\Cezan\AppData\Local\Temp\ipykernel_17672\837986230.py:23: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(path).astype(np.float32) / 255.0


Test #1 failed
   Expected center: [1.0, 1.0, 1.0]
   Actual center: [1.0, 1.0, 1.0]
   Max diff: 0.003921568393707275

Running Test Case #2: Small deterministic test for brightening
Test #2 failed
   Expected center: 0.699999988079071
   Actual center: 0.6990196108818054
   Max diff: 0.003921568393707275

Running Test Case #3: Edge case with values close to 1
Test #3 failed
   Expected center: [0.800000011920929, 1.0, 1.0]
   Actual center: [0.800000011920929, 0.9960784316062927, 1.0]
   Max diff: 0.8039215803146362


TypeError: Object of type float32 is not JSON serializable